<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE6-FaultPredictionAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:


# ============================================================
# Fault Classification Pipeline
# ============================================================

############PASTE ADAPTIVE WINDOW HERE - so everything is in one file - later, we can import as a package#####################


# ====== AdaptiveWindowAgent ======
# =====================================================
# AdaptiveWindowAgent (improved version)
# =====================================================
import numpy as np
import pandas as pd
import pickle, os, logging, datetime as dt
from collections import deque
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.vector_ar.var_model import VAR
import keras

logger = logging.getLogger(__name__)

class AdaptiveWindowAgent:
    """
    Adaptive Window Agent:
    - Predicts window size using MLP
    - Evaluates forecast with VAR
    - Monitors anomalies & drift with adaptive thresholds
    - Outputs severity scores + suppresses redundant events
    """

    def __init__(self, agent_id="adaptive_window_agent",
                 model_path=None, checkpoint_path=None):
        self.agent_id = agent_id
        self.model_path = model_path or "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
        self.checkpoint_path = checkpoint_path

        # Core model
        self.model = None
        self.transformer = StandardScaler()
        self.transformer_fitted = False
        self.is_model_loaded = False

        # Histories
        self.prediction_history = deque(maxlen=1000)
        self.mse_history = deque(maxlen=200)
        self.mae_history = deque(maxlen=200)

        # Event detection params
        self.drift_detection_window = 20
        self.drift_threshold_mse = 1.5   # stricter
        self.drift_threshold_mae = 1.5
        self.consecutive_poor_predictions = 0
        self.cooldown_counter = 0

        # Stats
        self.performance_stats = {
            'total_predictions': 0,
            'avg_mse': 0.0,
            'avg_mae': 0.0,
            'last_retrain_time': None,
            'drift_events': 0,
            'anomaly_events': 0,
            'retraining_events': 0
        }

        # Retraining buffers
        self.retraining_data = {
            'x_buffer': deque(maxlen=10000),
            'y_buffer': deque(maxlen=10000)
        }

        self.load_model()
        print(f"AdaptiveWindowAgent {self.agent_id} initialized")

    # ------------------- Model -------------------

    def load_model(self):
        try:
            if os.path.exists(self.model_path):
                self.model = keras.models.load_model(self.model_path)
                self.is_model_loaded = True
                print(f"✅ Loaded MLP model from {self.model_path}")

                # Try to load transformer
                transformer_path = self.model_path.replace('.keras', '_transformer.pkl')
                if os.path.exists(transformer_path):
                    with open(transformer_path, 'rb') as f:
                        self.transformer = pickle.load(f)
                    self.transformer_fitted = True
                else:
                    # Fit transformer from true window labels
                    y_original = np.load(
                        "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy"
                    )
                    self.transformer.fit(y_original.reshape(-1, 1))
                    self.transformer_fitted = True
                    with open(transformer_path, 'wb') as f:
                        pickle.dump(self.transformer, f)
                    print("⚠️ No transformer found, fitted a new one.")
            else:
                print(f"❌ Model not found at {self.model_path}")
        except Exception as e:
            print(f"❌ Error loading model: {e}")

    # ------------------- Forecast Eval -------------------
    def evaluate_forecast_performance(self, sequence_3d, predicted_window, n_future=1):
        try:
            df = pd.DataFrame(sequence_3d, columns=[f'V{i+1}' for i in range(sequence_3d.shape[1])])
            df_train, df_test = df[:-n_future], df[-n_future:]

            # Drop constant cols
            constant_cols = [c for c in df_train.columns if df_train[c].nunique() <= 1]
            df_train = df_train.drop(columns=constant_cols, errors="ignore")
            df_test = df_test.drop(columns=constant_cols, errors="ignore")

            # If too few variables, fall back immediately
            if len(df_train.columns) < 1:
                return self._persistence_forecast(df, df_test)

            k = min(predicted_window, len(df_train) - 2)
            if k < 1: k = 1

            # Try VAR
            try:
                model = VAR(df_train)
                model_fitted = model.fit(maxlags=k, trend="c")
                forecast_input = df_train.values[-model_fitted.k_ar:]
                fc = model_fitted.forecast(y=forecast_input, steps=n_future)
                df_forecast = pd.DataFrame(fc, index=df.index[-n_future:], columns=df_train.columns)

                actual = df_test[df_forecast.columns].values.flatten()
                predicted = df_forecast.values.flatten()

            except Exception:
                # Try AutoReg
                try:
                    from statsmodels.tsa.ar_model import AutoReg
                    col = df_train.columns[0]
                    model = AutoReg(df_train[col], lags=min(k, len(df_train)//2)).fit()
                    predicted = model.predict(start=len(df_train), end=len(df_train)+n_future-1).values
                    actual = df_test[col].values
                except Exception:
                    # Fallback to persistence
                    return self._persistence_forecast(df, df_test)

            mse = np.mean((actual - predicted) ** 2)
            mae = np.mean(np.abs(actual - predicted))

            # If forecast is unstable, fallback
            if np.isnan(mse) or np.isnan(mae) or mse > 10 or mae > 10:
                return self._persistence_forecast(df, df_test)

            return {
                'mse': float(mse),
                'mae': float(mae),
                'forecast_success': True,
                'actual_values': actual.tolist(),
                'predicted_values': predicted.tolist(),
                'used_columns': list(df_test.columns)
            }

        except Exception:
            return self._persistence_forecast(df, df_test)

# ------------------- Persistence fallback -------------------

    def _persistence_forecast(self, df, df_test):
        """Simple last-value-carried-forward forecast."""
        try:
            last_values = df.iloc[-1].values
            predicted = np.tile(last_values, (len(df_test), 1))
            actual = df_test.values.flatten()

            mse = np.mean((actual - predicted.flatten()) ** 2)
            mae = np.mean(np.abs(actual - predicted.flatten()))

            return {
                'mse': float(mse),
                'mae': float(mae),
                'forecast_success': True,
                'actual_values': actual.tolist(),
                'predicted_values': predicted.flatten().tolist(),
                'used_columns': list(df_test.columns),
                'note': 'persistence_fallback'
            }
        except Exception as e:
            return {'mse': 9999, 'mae': 9999, 'forecast_success': True, 'error': str(e), 'note': 'persistence_fallback_failed'}



    # ------------------- Prediction -------------------

    def predict_window_size(self, feature_vector, sequence_3d):
        if not self.is_model_loaded:
            return {'predicted_window': 20, 'error': "Model not loaded"}

        try:
            if feature_vector.ndim == 1:
                feature_vector = feature_vector.reshape(1, -1)

            pred_raw = self.model.predict(feature_vector, verbose=0)
            if self.transformer_fitted:
                predicted_window = int(round(self.transformer.inverse_transform(pred_raw)[0, 0]))
            else:
                predicted_window = int(round(pred_raw[0, 0]))

            # Evaluate
            forecast_metrics = self.evaluate_forecast_performance(sequence_3d, predicted_window, n_future=1)

            if forecast_metrics.get("forecast_success", False):
                self.mse_history.append(forecast_metrics["mse"])
                self.mae_history.append(forecast_metrics["mae"])
                self.performance_stats['total_predictions'] += 1
                self.performance_stats['avg_mse'] = np.mean(self.mse_history)
                self.performance_stats['avg_mae'] = np.mean(self.mae_history)

            # Event check
            event, sev = self._check_for_event()

            # Save history
            record = {
                'timestamp': dt.datetime.now(),
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'event_type': event,
                'severity': sev
            }
            self.prediction_history.append(record)

            return {
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'event_type': event,
                'severity': sev,
                'performance_stats': self.get_recent_performance()
            }
        except Exception as e:
            return {'predicted_window': 20, 'error': str(e)}

    # ------------------- Event Logic -------------------

    def _check_for_event(self):
        if len(self.mse_history) < self.drift_detection_window:
            return None, 0.0

        mse_vals = np.array(self.mse_history)[-self.drift_detection_window:]
        mae_vals = np.array(self.mae_history)[-self.drift_detection_window:]

        # Rolling stats
        mean_mse, std_mse = np.mean(mse_vals), np.std(mse_vals) + 1e-8
        last_mse = mse_vals[-1]

        # Normalized error
        norm_error = (last_mse - mean_mse) / std_mse

        # Severity
        anomaly_severity = max(0, norm_error)
        drift_severity = max(0, (np.mean(mse_vals) / (np.median(mse_vals)+1e-5)) - 1)

        # Check anomaly
        if last_mse > mean_mse + 2.0 * std_mse:
            self.performance_stats['anomaly_events'] += 1
            return "ANOMALY", anomaly_severity

        # Check drift (with persistence + cooldown)
        ema_mse = 0.3*np.mean(mse_vals) + 0.7*np.median(mse_vals)
        ema_mae = 0.3*np.mean(mae_vals) + 0.7*np.median(mae_vals)
        mse_ratio = ema_mse / max(np.median(mse_vals), 1e-5)
        mae_ratio = ema_mae / max(np.median(mae_vals), 1e-5)

        if mse_ratio > self.drift_threshold_mse and mae_ratio > self.drift_threshold_mae:
            self.consecutive_poor_predictions += 1
            if self.consecutive_poor_predictions >= 5 and self.cooldown_counter == 0:
                self.performance_stats['drift_events'] += 1
                self.cooldown_counter = 10
                return "DRIFT", drift_severity
        else:
            self.consecutive_poor_predictions = 0

        if self.cooldown_counter > 0:
            self.cooldown_counter -= 1

        return None, 0.0

    # ------------------- Helpers -------------------

    def get_recent_performance(self):
        successful_predictions = [
            p for p in list(self.prediction_history)[-50:]
            if p.get('forecast_metrics', {}).get('forecast_success', False)
        ]
        return {
            'total_predictions': len(self.prediction_history),
            'successful_predictions': len(successful_predictions),
            'success_rate': len(successful_predictions) / max(len(self.prediction_history), 1),
            'drift_events': self.performance_stats['drift_events'],
            'anomaly_events': self.performance_stats['anomaly_events'],
            'retraining_events': self.performance_stats['retraining_events'],
            'recent_mse': float(np.mean(list(self.mse_history)[-10:])) if self.mse_history else 0,
            'avg_mse': float(np.mean(self.mse_history)) if self.mse_history else 0,
            'recent_mae': float(np.mean(list(self.mae_history)[-10:])) if self.mae_history else 0,
            'avg_mae': float(np.mean(self.mae_history)) if self.mae_history else 0,
            'transformer_fitted': self.transformer_fitted
        }


    def save_performance_state(self, filepath: str):
        """Save performance statistics + prediction history to JSON"""
        try:
            state = {
                'performance_stats': self.performance_stats.copy(),
                'prediction_history': list(self.prediction_history)[-100:],  # last 100
                'mse_history': list(self.mse_history),
                'mae_history': list(self.mae_history),
                'transformer_fitted': self.transformer_fitted
            }
            import json
            with open(filepath, 'w') as f:
                json.dump(state, f, indent=2, default=str)
            print(f"✅ Performance state saved to {filepath}")
        except Exception as e:
            print(f"❌ Failed to save performance state: {e}")


#############################################
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib

# =============================================================================================================================
# 1. Load Data - BASELINE MODELS----------------------------
# ============================================================-======================================================================
# Long subsequences (length=50) and labels
long_sequences = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy")
labels_detection = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_detection.npy")  # fault labels
labels_h1 = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_H1.npy")  # optional predictive labels

H = 1500  # size of holdout (future block)
X_holdout = long_sequences[-H:]
y_holdout = labels_detection[-H:]

long_sequences = long_sequences[:-H]
labels_detection = labels_detection[:-H]

print("Data loaded:", long_sequences.shape, labels_detection.shape)

# ============================================================
# 2. Baseline 1 Fixed Window Features (window=50, flattened)
# ============================================================
X_fixed = long_sequences.reshape(long_sequences.shape[0], -1)
y = labels_detection  # detection labels (can switch to h1/h3/h12)

X_train, X_test, y_train, y_test = train_test_split(X_fixed, y, test_size=0.2, random_state=42, stratify=y)

print("Train/Test shapes:", X_train.shape, X_test.shape)

# Baseline-1: Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\n=== Baseline-1: Fixed Window Random Forest test data ===")
print(classification_report(y_test, y_pred_rf, digits=4))
print(confusion_matrix(y_test, y_pred_rf))
joblib.dump(rf, "rf_fixed_window.pkl")

########## HOLDOUT
X_fixed_h = X_holdout.reshape(X_holdout.shape[0], -1)
y_predh_rf = rf.predict(X_fixed_h)

print("\n=== Baseline-1: Fixed Window Random Forest holdout data ===")
print(classification_report(y_holdout, y_predh_rf, digits=4))
print(confusion_matrix(y_holdout, y_predh_rf))
joblib.dump(rf, "rf_fixed_window_holdout.pkl")


# ==========================================================================================================================
# 3. Dynamic Window Features (using your AdaptiveWindowAgent)
# =============================================================================================================================
#from agents.adaptive_window_agent import AdaptiveWindowAgent

window_agent = AdaptiveWindowAgent(
    model_path="/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
)

##Load data again, fresh

long_sequences = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy")
labels_detection = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_detection.npy")  # fault labels
labels_h1 = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_H1.npy")  # optional predictive labels



# Generate features for each subsequence using predicted window - including holdout.
import os
import numpy as np

# Paths
dyn_feat_path = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/dynamic_features.npy"

if os.path.exists(dyn_feat_path):
    print(f"✅ Loading existing dynamic features from {dyn_feat_path}")
    dynamic_features = np.load(dyn_feat_path, allow_pickle=True)
else:
    print("⚠️ Dynamic features not found. Generating now...")

    dynamic_features = []
    for i, seq in enumerate(long_sequences):
        try:
            features = seq.flatten()
            result = window_agent.predict_window_size(features, seq)
            w = result.get("predicted_window", 50)

            # Extract last w timesteps from the long sequence
            seq_dynamic = seq[-w:].flatten()
            dynamic_features.append(seq_dynamic)

            if (i + 1) % 100 == 0:
                print(f"Processed {i+1}/{len(long_sequences)} sequences...")

        except Exception as e:
            print(f"❌ Error on sequence {i}: {e}")
            # fallback: use fixed window
            seq_dynamic = seq[-50:].flatten()
            dynamic_features.append(seq_dynamic)

    dynamic_features = np.array(dynamic_features, dtype=object)
    np.save(dyn_feat_path, dynamic_features)
    print(f"✅ Saved dynamic features to {dyn_feat_path}")

print("Dynamic feature shape:", np.shape(dynamic_features))

# Pad to same length (use max window=50)
X_dynamic = np.array([np.pad(f, (0, 50*seq.shape[1] - len(f))) for f in dynamic_features])
H = 1500  # size of holdout (future block)
X_holdout = X_dynamic[-H:]
y_holdout = labels_detection[-H:]

long_sequences = X_dynamic[:-H]
labels_detection = labels_detection[:-H]

X_train_dyn, X_test_dyn, y_train_dyn, y_test_dyn = train_test_split(long_sequences, labels_detection, test_size=0.2, random_state=42, stratify=labels_detection)

# Baseline-2: Random Forest + dynamic window - test data
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_dyn, y_train_dyn)
y_pred_rf_dyn = rf.predict(X_test_dyn)

print("\n=== Baseline-2: dynamic Window Random Forest ===")
print(classification_report(y_test_dyn, y_pred_rf_dyn, digits=4))
print(confusion_matrix(y_test_dyn, y_pred_rf_dyn))
joblib.dump(rf, "rf_dynamic_window.pkl")


########## Baseline-2: Random Forest + dynamic window -HOLDOUT

y_predh_rf_dyn_h = rf.predict(X_holdout)

print("\n=== Baseline-2: dynamic Window Random Forest holdout data ===")
print(classification_report(y_holdout, y_predh_rf_dyn_h, digits=4))
print(confusion_matrix(y_holdout, y_predh_rf_dyn_h))
joblib.dump(rf, "rf_dynamic_window_holdout.pkl")


# Baseline-3: XGBoost dynamic
xgb = XGBClassifier(
    n_estimators=300, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, n_jobs=-1
)
xgb.fit(X_train_dyn, y_train_dyn)
y_pred_xgb = xgb.predict(X_test_dyn)

print("\n=== Baseline-3: Dynamic Window XGBoost ===")
print(classification_report(y_test_dyn, y_pred_xgb, digits=4))
print(confusion_matrix(y_test_dyn, y_pred_xgb))
joblib.dump(xgb, "xgb_dynamic_window.pkl")

########## Baseline-3: XGBoost dynamic HOLDOUT

y_predh_xgb_h = xgb.predict(X_holdout)

print("\n=== Baseline-3:Dynamic Window XGBoost holdout data ===")
print(classification_report(y_holdout, y_predh_xgb_h, digits=4))
print(confusion_matrix(y_holdout, y_predh_xgb_h))
joblib.dump(xgb, "xgb_dynamic_window_holdout.pkl")


# ============================================================
# Ready for Next Steps
# ============================================================
print("\n✅ Baseline models trained and evaluated.")
print("Next: add Transformer-based sequence classifier, then coordinator features.")



# ========================================================================================
# SOTA: Transformer (PatchTST-style) Classifier
# =======================================================================================
# ========================================================================================
# Contrastive Pretraining + Transformer Encoder + kNN Classifier
# ========================================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# ====================
# 1. Mixed Precision
# ====================
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# ====================
# 2. Transformer Block
# ====================
def transformer_block(inputs, head_size=32, num_heads=2, ff_dim=64, dropout=0.2):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    out = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return out

# ====================
# 3. Transformer Encoder (for contrastive pretrain)
# ====================
def build_transformer_encoder(input_shape, projection_dim=64):
    inputs = keras.Input(shape=input_shape)
    x = transformer_block(inputs, 32, 2, 64, 0.2)
    x = transformer_block(x, 32, 2, 64, 0.2)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(projection_dim, activation=None)(x)  # projection head
    return keras.Model(inputs, outputs, name="transformer_encoder")

# ====================
# 4. Contrastive Loss (NT-Xent style)
# ====================
def contrastive_loss(temperature=0.1):
    def loss(y_true, y_pred):
        y_pred = K.l2_normalize(y_pred, axis=-1)
        batch_size = tf.shape(y_pred)[0] // 2
        z1, z2 = tf.split(y_pred, 2, axis=0)
        sim = tf.matmul(z1, z2, transpose_b=True) / temperature
        labels = tf.range(batch_size)
        loss1 = keras.losses.sparse_categorical_crossentropy(labels, sim, from_logits=True)
        loss2 = keras.losses.sparse_categorical_crossentropy(labels, tf.transpose(sim), from_logits=True)
        return tf.reduce_mean(loss1 + loss2)
    return loss

# ====================
# 5. Augmentations for contrastive
# ====================
def augment_sequence(seq):
    noise = np.random.normal(0, 0.01, seq.shape)
    scale = np.random.uniform(0.9, 1.1)
    mask = np.random.binomial(1, 0.95, seq.shape)
    return seq * scale + noise * mask

def create_contrastive_pairs(X, batch_size=128):
    idx = np.random.choice(len(X), batch_size, replace=False)
    x1 = np.array([augment_sequence(X[i]) for i in idx])
    x2 = np.array([augment_sequence(X[i]) for i in idx])
    return np.concatenate([x1, x2], axis=0)

# ====================
# 6. Load Data + Holdout
# ====================
long_sequences = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy")
labels_detection = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_detection.npy")

H = 2000   # holdout size (future block, more faults)
X_holdout = long_sequences[-H:]
y_holdout = labels_detection[-H:]
X = long_sequences[:-H]
y = labels_detection[:-H]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)
print("Holdout:", X_holdout.shape, y_holdout.shape)

# class weights
classes = np.unique(y_train)
cw = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = {i: w for i, w in zip(classes, cw)}
print("Class weights:", class_weight_dict)

# ====================
# 7. Contrastive Pretraining
# ====================
encoder = build_transformer_encoder(X_train.shape[1:], projection_dim=64)
contrastive_model = keras.Model(encoder.input, encoder.output)
contrastive_model.compile(optimizer=keras.optimizers.Adam(1e-3),
                          loss=contrastive_loss(temperature=0.1))

EPOCHS = 10
BATCH_SIZE = 128
for epoch in range(EPOCHS):
    X_batch = create_contrastive_pairs(X_train, batch_size=BATCH_SIZE//2)
    y_dummy = np.zeros(len(X_batch))
    loss = contrastive_model.train_on_batch(X_batch, y_dummy)
    if epoch % 2 == 0:
        print(f"Epoch {epoch}: contrastive loss = {loss:.4f}")

# ====================
# 8. Build Classifier (fine-tune encoder)
# ====================
encoder_backbone = build_transformer_encoder(X_train.shape[1:], projection_dim=64)
encoder_backbone.set_weights(encoder.get_weights())  # transfer pretrained weights

inputs = keras.Input(shape=X_train.shape[1:])
x = encoder_backbone(inputs)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid", dtype="float32")(x)

classifier = keras.Model(inputs, outputs)

# focal loss
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        bce = K.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        modulating = (1 - p_t) ** gamma
        return K.mean(alpha_factor * modulating * bce, axis=-1)
    return loss

classifier.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=focal_loss(),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall")]
)

early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
lr_sched = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

history = classifier.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=128,
    class_weight=class_weight_dict,
    callbacks=[early_stop, lr_sched],
    verbose=1
)

# ====================
# 9. Evaluate on val
# ====================
y_probs = classifier.predict(X_val).flatten()
y_pred = (y_probs >= 0.5).astype(int)
print("\n=== Contrastive Transformer (Validation) ===")
print(classification_report(y_val, y_pred, digits=4))
print(confusion_matrix(y_val, y_pred))

# ====================
# 10. Evaluate on holdout
# ====================
y_probs = classifier.predict(X_holdout).flatten()
y_pred = (y_probs >= 0.5).astype(int)
print("\n=== Contrastive Transformer (Holdout) ===")
print(classification_report(y_holdout, y_pred, digits=4))
print(confusion_matrix(y_holdout, y_pred))
#######################################################################################################
#Validate with coordination agent###############################
#######################################################################################################

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. Validation Function
# --------------------------
def validate_with_coordinator(y_probs, y_true, coordinator_results, threshold=0.5):
    """
    Validate Transformer predictions against coordinator agent decisions.
    """
    # Transformer raw predictions
    y_pred_raw = (y_probs >= threshold).astype(int)

    # Apply coordinator filter
    y_pred_validated = []
    for pred, coord in zip(y_pred_raw, coordinator_results):
        coord_flag = coord["coordinator"].get("final_anomaly", False)
        if pred == 1 and coord_flag:
            y_pred_validated.append(1)   # confirmed
        else:
            y_pred_validated.append(0)   # suppress FP or keep 0

    y_pred_validated = np.array(y_pred_validated)

    # Reports
    report_raw = classification_report(y_true, y_pred_raw, output_dict=True, zero_division=0)
    report_validated = classification_report(y_true, y_pred_validated, output_dict=True, zero_division=0)

    cm_raw = confusion_matrix(y_true, y_pred_raw)
    cm_validated = confusion_matrix(y_true, y_pred_validated)

    return {
        "report_raw": report_raw,
        "report_validated": report_validated,
        "confusion_matrix_raw": cm_raw,
        "confusion_matrix_validated": cm_validated,
        "raw_preds": y_pred_raw,
        "validated_preds": y_pred_validated
    }

# --------------------------
# 2. Confusion Matrix Plot
# --------------------------
def plot_confusion_matrices(cm_raw, cm_validated, labels=[0,1]):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sns.heatmap(cm_raw, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels, ax=axes[0])
    axes[0].set_title("Transformer (Raw)")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")

    sns.heatmap(cm_validated, annot=True, fmt="d", cmap="Greens",
                xticklabels=labels, yticklabels=labels, ax=axes[1])
    axes[1].set_title("Transformer + Coordinator")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True")

    plt.tight_layout()
    plt.show()

# --------------------------
# 3. Run Coordinator on Test Set
# --------------------------
results = []
coordinator_flags = []

print("Running Coordinator over test set...")

for i, seq in enumerate(X_test_long):   # X_test_long must match X_val / y_val split
    try:
        features = seq.flatten()

        master_out = master.process_system_input(seq)
        window_out = window_agent.predict_window_size(features, seq)
        final = coordinator.fuse(master_out, window_out)

        results.append({
            "master": master_out,
            "window": window_out,
            "coordinator": final
        })

        coordinator_flags.append(1 if final["final_anomaly"] else 0)

    except Exception as e:
        print(f"Sample {i} failed: {e}")
        results.append({"coordinator": {"final_anomaly": False}})
        coordinator_flags.append(0)

coordinator_flags = np.array(coordinator_flags)
print(f"✅ Coordinator outputs generated: {coordinator_flags.shape}")
print(f"Unique flags: {np.unique(coordinator_flags, return_counts=True)}")

# --------------------------
# 4. Validate Predictions
# --------------------------
y_probs = model.predict(X_test_long).flatten()   # Transformer predictions
y_true  = y_test                                # Ground truth fault labels

validation_out = validate_with_coordinator(
    y_probs=y_probs,
    y_true=y_true,
    coordinator_results=results,
    threshold=0.5
)

print("\n=== Transformer (Raw) ===")
print(classification_report(y_true, validation_out["raw_preds"], digits=4))
print(validation_out["confusion_matrix_raw"])

print("\n=== Transformer + Coordinator (Validated) ===")
print(classification_report(y_true, validation_out["validated_preds"], digits=4))
print(validation_out["confusion_matrix_validated"])

# --------------------------
# 5. Coordinator Impact Summary
# --------------------------
raw = validation_out["report_raw"]["1"]
val = validation_out["report_validated"]["1"]

print("\n=== Coordinator Impact on Fault Class (1) ===")
print(f"Precision: {raw['precision']:.3f} → {val['precision']:.3f}")
print(f"Recall:    {raw['recall']:.3f} → {val['recall']:.3f}")
print(f"F1-score:  {raw['f1-score']:.3f} → {val['f1-score']:.3f}")

# --------------------------
# 6. Plot Confusion Matrices
# --------------------------
plot_confusion_matrices(
    validation_out["confusion_matrix_raw"],
    validation_out["confusion_matrix_validated"]
)

# ==========================================
# 7. Overlay Plot: Transformer vs Coordinator vs Ground Truth
# ==========================================
def plot_timeline(y_true, raw_preds, validated_preds, coordinator_flags, max_samples=200):
    """
    Visual timeline of ground truth faults, transformer predictions, and coordinator validated results.
    """
    n = min(max_samples, len(y_true))
    t = np.arange(n)

    plt.figure(figsize=(14, 5))

    # Ground truth faults
    plt.plot(t, y_true[:n], label="Ground Truth", color="black", linewidth=2, alpha=0.7)

    # Transformer raw predictions
    plt.plot(t, raw_preds[:n], label="Transformer Raw", color="red", linestyle="--", alpha=0.6)

    # Coordinator validated predictions
    plt.plot(t, validated_preds[:n], label="Validated (Transformer+Coordinator)",
             color="green", linewidth=2, alpha=0.7)

    # Coordinator anomaly flags only
    plt.scatter(t, coordinator_flags[:n], label="Coordinator Final Anomaly",
                marker="x", color="blue", alpha=0.8)

    plt.xlabel("Sample Index")
    plt.ylabel("Fault / Anomaly Flag")
    plt.title("Timeline: Transformer vs Coordinator vs Ground Truth")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# ==========================================
# Call Timeline Plot
# ==========================================
plot_timeline(
    y_true=y_true,
    raw_preds=validation_out["raw_preds"],
    validated_preds=validation_out["validated_preds"],
    coordinator_flags=coordinator_flags,
    max_samples=200
)

# ==========================================
# 8. Quantify False Positive Suppression
# ==========================================
def analyze_false_positive_suppression(y_true, raw_preds, validated_preds):
    """
    Compare raw vs validated predictions to quantify false positive suppression.
    """
    # False positives = predicted 1, but true = 0
    fp_raw = np.sum((raw_preds == 1) & (y_true == 0))
    fp_validated = np.sum((validated_preds == 1) & (y_true == 0))

    # False negatives = predicted 0, but true = 1
    fn_raw = np.sum((raw_preds == 0) & (y_true == 1))
    fn_validated = np.sum((validated_preds == 0) & (y_true == 1))

    suppression_rate = (fp_raw - fp_validated) / max(fp_raw, 1)

    return {
        "false_positives_raw": int(fp_raw),
        "false_positives_validated": int(fp_validated),
        "false_negatives_raw": int(fn_raw),
        "false_negatives_validated": int(fn_validated),
        "suppression_rate": suppression_rate
    }

# Run analysis
fp_analysis = analyze_false_positive_suppression(
    y_true=y_true,
    raw_preds=validation_out["raw_preds"],
    validated_preds=validation_out["validated_preds"]
)

print("\n=== False Positive Suppression Analysis ===")
print(f"False Positives (Raw Transformer): {fp_analysis['false_positives_raw']}")
print(f"False Positives (Validated): {fp_analysis['false_positives_validated']}")
print(f"False Negatives (Raw Transformer): {fp_analysis['false_negatives_raw']}")
print(f"False Negatives (Validated): {fp_analysis['false_negatives_validated']}")
print(f"Suppression Rate: {fp_analysis['suppression_rate']*100:.2f}%")


Train shape: (901, 50, 12) (901,)
Val shape: (226, 50, 12) (226,)
Epoch 0: contrastive loss = 6.6508
Epoch 2: contrastive loss = 6.3960
Epoch 4: contrastive loss = 6.0980
Epoch 6: contrastive loss = 5.7791
Epoch 8: contrastive loss = 5.5074
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step

=== Contrastive Transformer + kNN (Validation) ===
              precision    recall  f1-score   support

           0     0.9822    1.0000    0.9910       221
           1     1.0000    0.2000    0.3333         5

    accuracy                         0.9823       226
   macro avg     0.9911    0.6000    0.6622       226
weighted avg     0.9826    0.9823    0.9765       226

[[221   0]
 [  4   1]]

=== Contrastive Transformer + kNN (Holdout) ===
              precision    recall  f1-score   support

           0     0.9738    0.9906    0.9821      2435
           1     0.0000    0.0000    0.0000        65

    accuracy             

In [ ]:
from google.colab import drive
drive.mount('/content/drive')